In [1]:
#Importing dependencies
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Function to preprocess images
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image

In [3]:
# Function to load and preprocess images
def load_and_preprocess_image(file_path):
    image = tf.io.read_file(file_path)
    return preprocess_image(image)

In [4]:
# Function to combine genres for each image
def combine_labels(data):
    combined_data = {}
    for _, row in data.iterrows():
        image_name = row['Image name']
        genre = row['Genre']
        if image_name in combined_data:
            combined_data[image_name].add(genre)
        else:
            combined_data[image_name] = {genre}
    return combined_data

In [5]:
# Function to load data from master folder (Train)
def load_data_from_master_folder(master_folder, image_dir, num_classes):
    csv_files = [os.path.join(master_folder, f) for f in os.listdir(master_folder) if f.endswith('.csv')]
    datasets = []

    # Initialize genre mapping dictionary
    genre_mapping = {}
    idx = 0

    # Update genre mapping if genre is not already mapped
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        for genre in df['Genre'].unique():
            if genre not in genre_mapping:
                genre_mapping[genre] = idx
                idx += 1

    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        combined_data = combine_labels(df)
        file_paths_labels = {}

        # Create file paths and multi-hot encoded labels
        for image_name, genres in combined_data.items():
            genre_list = list(genres)
            file_path = os.path.join(image_dir, genre_list[0], image_name)
            if file_path not in file_paths_labels:
                file_paths_labels[file_path] = np.zeros(num_classes)
            for genre in genres:
                file_paths_labels[file_path][genre_mapping[genre]] = 1

        # Convert file paths and labels to lists
        file_paths = list(file_paths_labels.keys())
        labels = np.array(list(file_paths_labels.values()))

        # Create TensorFlow dataset
        path_ds = tf.data.Dataset.from_tensor_slices(file_paths)
        image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        label_ds = tf.data.Dataset.from_tensor_slices(labels.astype('float32'))
        dataset = tf.data.Dataset.zip((image_ds, label_ds)).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
        datasets.append(dataset)

    return datasets, len(csv_files), genre_mapping


In [6]:
# Function to load data from master folder (Validation and Test)
def load_data_with_existing_genre_mapping(master_folder, image_dir, num_classes, genre_mapping):
    csv_files = [os.path.join(master_folder, f) for f in os.listdir(master_folder) if f.endswith('.csv')]
    datasets = []

    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        combined_data = combine_labels(df)
        file_paths_labels = {}

        for image_name, genres in combined_data.items():
            genre_list = list(genres)
            file_path = os.path.join(image_dir, genre_list[0], image_name)
            if file_path not in file_paths_labels:
                file_paths_labels[file_path] = np.zeros(num_classes)
            for genre in genres:
                if genre in genre_mapping:
                    file_paths_labels[file_path][genre_mapping[genre]] = 1

        file_paths = list(file_paths_labels.keys())
        labels = np.array(list(file_paths_labels.values()))

        path_ds = tf.data.Dataset.from_tensor_slices(file_paths)
        image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        label_ds = tf.data.Dataset.from_tensor_slices(labels.astype('float32'))
        dataset = tf.data.Dataset.zip((image_ds, label_ds)).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
        datasets.append(dataset)

    return datasets

In [7]:
# CNN Architecture (Think about adding batch normalization)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    # model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# CNN Architecture (Think about adding batch normalization)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(20, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 220, 220, 16)      1216      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 110, 110, 16)     0         
 2D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 110, 110, 16)     64        
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, 106, 106, 32)      12832     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 53, 53, 32)       0         
 2D)                                                             
                                                      

In [6]:
plot_model(model, to_file='model.png')

NameError: name 'plot_model' is not defined

In [8]:
#Enhancement of cluster structuring
from sklearn.neighbors import NearestNeighbors

def neighbor_discriminant_term(features, labels, k):
    features_np = features.numpy()
    labels_np = labels.numpy()

    # Dynamically set k_neighbors based on available samples
    num_samples = features_np.shape[0]
    k_neighbors = min(k, num_samples - 1)
    
    if k_neighbors < 1:
        return 0.0

    # Calculate pairwise distances
    nbrs = NearestNeighbors(n_neighbors=k_neighbors+1, algorithm='auto').fit(features_np)
    distances, indices = nbrs.kneighbors(features_np)

    discriminant_loss = 0.0

    for i, neighbors in enumerate(indices):
        # Skip the first neighbor since it's the point itself
        neighbors = neighbors[1:]
        same_label_count = 0
        diff_label_count = 0

        for neighbor in neighbors:
            if np.array_equal(labels_np[i], labels_np[neighbor]):
                same_label_count += 1
            else:
                diff_label_count += 1
        
        if same_label_count + diff_label_count > 0:
            discriminant_loss += diff_label_count / (same_label_count + diff_label_count)

    return discriminant_loss / num_samples

In [9]:
# Updated correlation_loss function to include necessary arguments (Encourages selection of complementary features)
def correlation_loss(models, lambda_weight, images, labels, loss_fn):
    total_loss = 0.0
    num_models = len(models)
    for i in range(num_models):
        for j in range(i + 1, num_models):
            with tf.GradientTape() as tape_i:
                predictions_i = models[i](images, training=True)
                loss_i = loss_fn(labels, predictions_i)
            gradients_i = tape_i.gradient(loss_i, models[i].trainable_variables)
            
            with tf.GradientTape() as tape_j:
                predictions_j = models[j](images, training=True)
                loss_j = loss_fn(labels, predictions_j)
            gradients_j = tape_j.gradient(loss_j, models[j].trainable_variables)
            
            dot_product = tf.reduce_sum([tf.reduce_sum(tf.multiply(grad_i, grad_j)) for grad_i, grad_j in zip(gradients_i, gradients_j)])
            norm_i = tf.reduce_sum([tf.norm(grad_i) for grad_i in gradients_i])
            norm_j = tf.reduce_sum([tf.norm(grad_j) for grad_j in gradients_j])
            
            correlation = dot_product / (norm_i * norm_j + 1e-8)
            total_loss += correlation
    return lambda_weight * total_loss

In [10]:
input_shape = (224, 224, 3)
num_epochs = 10
all_models = []
lambda_weight = 0.001
k_neighbors = 5

In [11]:
# Load Train data
master_directory = "/home/prolay/Desktop/Kartavya Desai/Folder_For_DFS"
image_master_directory = '/home/prolay/Desktop/Kartavya Desai/Augmented image_two_three/Train_TwoThree_Aug'
num_classes = 20  # Adjust this number based on the total number of unique genres

In [12]:
datasets, num_subsets, genre_mapping = load_data_from_master_folder(master_directory, image_master_directory, num_classes)

In [13]:
# Load Validation data
validation_master_directory = '/home/prolay/Desktop/Kartavya Desai/Validation CSV'
validation_image_master_directory = '/home/prolay/Desktop/Kartavya Desai/Augmented image_two_three/Val_TwoThree_Aug'

In [14]:
val_datasets = load_data_with_existing_genre_mapping(validation_master_directory, validation_image_master_directory, num_classes, genre_mapping)

In [15]:
# from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

# # Callbacks for storing training logs and checkpoints
# checkpoint_dir = "./checkpoints"
# os.makedirs(checkpoint_dir, exist_ok=True)
# checkpoint_callback = ModelCheckpoint(
#     filepath=os.path.join(checkpoint_dir, "model_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}.h5"),
#     save_best_only=True,
#     monitor='val_accuracy',
#     mode='max',
#     verbose=1
# )
# csv_logger = CSVLogger('training_log.csv', append=True)

# # Prepare the list of callbacks
# callbacks = [checkpoint_callback, csv_logger]

In [ ]:
# import matplotlib.pyplot as plt
# import tensorflow as tf
# import numpy as np

# Initialize lists to store metrics
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

# Training loop with shared weights, correlation loss, and validation accuracy
for i, dataset in enumerate(datasets):
    print(f"Training on subset {i+1}/{num_subsets}")
    
    model = create_model(input_shape, num_classes)
    if i > 0:
        model.set_weights(all_models[-1].get_weights())

    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.BinaryCrossentropy()

    for epoch in range(num_epochs):
        epoch_loss_avg = tf.keras.metrics.Mean()
        train_accuracy = tf.keras.metrics.BinaryAccuracy()

        # Training step
        for images, labels in dataset:
            with tf.GradientTape() as tape:
                predictions = model(images, training=True)
                loss_value = loss_fn(labels, predictions)
                if len(all_models) > 0:
                    loss_value += correlation_loss(all_models, lambda_weight, images, labels, loss_fn)
                
                # Neighbor-based discriminant term
                features = model(images, training=False)
                discriminant_loss = neighbor_discriminant_term(features, labels, k_neighbors)
                total_loss = loss_value + discriminant_loss

            gradients = tape.gradient(total_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            epoch_loss_avg.update_state(total_loss)
            train_accuracy.update_state(labels, predictions)

        # Record training loss and accuracy
        train_losses.append(epoch_loss_avg.result())
        train_accuracies.append(train_accuracy.result())

        print(f"Epoch {epoch + 1}, Loss: {epoch_loss_avg.result()}, Accuracy: {train_accuracy.result()}")

        # Validation step
        val_loss_avg = tf.keras.metrics.Mean()
        val_accuracy = tf.keras.metrics.BinaryAccuracy()
        for val_images, val_labels in val_datasets[i]:
            val_predictions = model(val_images, training=False)
            val_loss = loss_fn(val_labels, val_predictions)
            val_loss_avg.update_state(val_loss)
            val_accuracy.update_state(val_labels, val_predictions)

        # Record validation loss and accuracy
        val_losses.append(val_loss_avg.result())
        val_accuracies.append(val_accuracy.result())

        print(f"Validation Loss: {val_loss_avg.result()}, Validation Accuracy: {val_accuracy.result()}")

    all_models.append(model)

Training on subset 1/20


2024-07-08 09:53:18.288883: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Loss: 0.8084489107131958, Accuracy: 0.9123438596725464


2024-07-08 09:54:13.938304: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Validation Loss: 0.7532879114151001, Validation Accuracy: 0.8940645456314087


In [ ]:
final_model = all_models[-1]

In [ ]:
final_model.save("completed_model.h5")

In [ ]:
# Load test data
test_master_directory = '/home/prolay/Desktop/Kartavya Desai/Test_DFS'
test_image_master_directory = "/home/prolay/Desktop/Kartavya Desai/Augmented image_two_three/Test_TwoThree_Aug"

In [ ]:
test_datasets = load_data_with_existing_genre_mapping(test_master_directory, test_image_master_directory, num_classes, genre_mapping)

In [ ]:
#Model Evaluation Function
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, precision_score, recall_score

def evaluate_model(model, test_dataset):
    y_true = []
    y_pred = []

    for images, labels in test_dataset:
        predictions = model.predict(images)
        y_true.extend(labels.numpy())
        y_pred.extend((predictions > 0.18).astype(int))

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    print(y_true.shape)
    print(y_pred.shape)
    print(y_true[:5])
    print(y_pred[:5])
    h_loss = hamming_loss(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='micro')
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')

    print(f"Hamming Loss: {h_loss}")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")

    return h_loss, accuracy, f1, precision, recall

In [ ]:
#Evaluation of all the subset.
for i in range(len(test_datasets)):
    evaluate_model(final_model, test_datasets[i])
  # Evaluating on the first subset of the test dataset